In [ ]:
import pywavefront

In [ ]:
# parameters
HEIGHT_MAX = 150
WIDTH_MAX = 100
OUTPUT_FILE = 'result.schematic'

# データ読み込み

In [ ]:
data = 'data/sample_data.obj'

In [ ]:
scene = pywavefront.Wavefront(data)

In [ ]:
len(scene.vertices)

In [ ]:
import pandas as pd

In [ ]:
coordinates = ['x','y','z']
colors = ['r','g','b']
df = pd.DataFrame(scene.vertices,columns=coordinates + colors)
df.describe()

# `(x,y,z)`座標を0起点に変換

In [ ]:
df[coordinates] = df[coordinates] - df[coordinates].apply(min)
'''
for coor in coordinates:
    df[coor] = df[coor] - min(df[coor])
'''
df[coordinates].describe()

# `(x,y,z)`座標の拡大倍率を決定する

In [ ]:
%%time
max_series = df[coordinates].apply(max)
max_val = max_series.max()
max_idx = max_series.idxmax()
'''
for coor in coordinates:
    if max_val < max(df[coor]):
        max_val = max(df[coor])
        max_idx = coor
'''
print('col:',max_idx)
print('max_val:',max_val)

In [ ]:
if max_idx in ['x','z']:
    zoom_ratio = WIDTH_MAX / max_val
else:
    zoom_ratio = HEIGHT_MAX / max_val
print('zoom_ratio:',zoom_ratio)

In [ ]:
df[coordinates] = df[coordinates] * zoom_ratio
#for coor in coordinates:
#    df[coor] = df[coor] * zoom_ratio
df[coordinates].describe()

# 外接矩形領域の終点

In [ ]:
bounds_e = df[coordinates].apply(max)
print(bounds_e)

# 色情報を復元
値が`1.0`の場合に最大値(`255`)となるように変換  
※これであってるのかは不明

In [ ]:
COLOR_MAX = 255
norm_colors = ['norm_r','norm_g','norm_b']
df[norm_colors] = df[colors] * COLOR_MAX
#for color in colors:
#    df[color] = df[color] * COLOR_MAX
df[colors + norm_colors].describe()

# 座標値を整数に変換（切り捨て）

In [ ]:
import numpy as np
df[coordinates] = df[coordinates].apply(np.floor).astype(int)
df[coordinates].describe()

# ボクセル位置を表す識別子用の列を追加

In [ ]:
def gen_coordinates_id(row):
    return ','.join([str(v) for v in row.values])

In [ ]:
%%time
coor_id = 'coordinate_id'
df[coor_id] = df[coordinates].apply(gen_coordinates_id,axis=1)
df[coor_id]

# 各ボクセルの色の代表値を決める

In [ ]:
%%time
from tqdm import tqdm
typical_colors = {}
for coor_id_val in tqdm(set(df[coor_id])):
    target = df[df[coor_id] == coor_id_val][colors + norm_colors]
    typical_colors[coor_id_val] = target.mean()
#typical_colors

In [ ]:
voxel = np.zeros(list((bounds_e + 1).apply(np.ceil).values.astype(int)) + [3])
voxel.shape

In [ ]:
for coor,color in tqdm(typical_colors.items()):
    x,y,z = [int(v) for v in coor.split(',')]
    voxel[x,y,z] = color[norm_colors].values

# 結果表示

In [ ]:
#%matplotlib inline
#%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.image import imread
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
fig = plt.figure(figsize=(8,20))
ax = fig.add_subplot(111, projection='3d')
for coor,color in tqdm(typical_colors.items()):
    x,y,z = [int(v) for v in coor.split(',')]
    ax.plot(x,z,y,marker='.',color=color[colors])
#ax.scatter3D(np.ravel(x), np.ravel(z), np.ravel(y),s=10,marker='.')
ax.set_title("result")
plt.show()

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go
import os
from plotly.offline import iplot, init_notebook_mode

In [ ]:
#import plotly.graph_objects as go
import numpy as np

# Helix equation
t = np.linspace(0, 10, 50)
x, y, z = np.cos(t), np.sin(t), t

fig = go.Figure(data=[go.Scatter3d(x=df['x'], y=df['y'], z=df['z'],
                                   mode='markers')])
fig.show()

# Schematicファイルに変換

In [ ]:
blocks = []
data = []
width,height,length = voxel.shape[:3]
print(width,height,length)

In [ ]:
def _packing(schematic_obj):
    schematic_obj.root['Blocks'] = tag.ByteArray(blocks)
    schematic_obj.root['Data'] = tag.ByteArray(data)
    schematic_obj.root['Width'] = tag.Short(width)
    schematic_obj.root['Length'] = tag.Short(length)
    schematic_obj.root['Height'] = tag.Short(height)

def _save_schematic(out_file,schematic_obj):
    schematic_obj.save(out_file)

def output(out_path,schematic_obj):
    _packing(schematic_obj)
    _save_schematic(out_path,schematic_obj)

In [ ]:
def calc_nearest_block(obj_color,config):
    best = float('inf')
    for c in config:
        p_color = c['COLOR']
        diff = obj_color - p_color
        diff_dist = sum(diff ** 2)
        if best > diff_dist:
            best = diff_dist
            color_data = c
            if best == 0:
                break
    return color_data


In [ ]:
# 設定ファイルを読み込み
import json
with open('config/block_info.json', 'r', encoding='utf-8') as f:
    config = json.load(f)
# config

## [Schematic File Format](https://minecraft.gamepedia.com/Schematic_file_format)

> Indices for the Blocks and Data arrays are ordered YZX - that is, the X coordinate varies the fastest. 

> Blocks: Block IDs defining the terrain. 8 bits per block. Sorted by height (bottom to top) then length then width—the index of the block at X,Y,Z is (Y×length + Z)×width + X.

`X`は`width`、`Y`は`height`、`Z`は`length`に対応する。  
`Blocks`と`Data`は`X`方向への走査後に、`Z`方向にうつり、その後`Y`方向に移動する。

In [ ]:
X,Y,Z = voxel.shape[:3]
for y in tqdm(range(Y)):
    for z in range(Z):
        for x in range(X):    
            if (voxel[x,y,z] == 0).all():
                blocks.append(0)
                data.append(0)
            else:
                color_data = calc_nearest_block(voxel[x,y,z],config)
                blocks.append(color_data['BLOCK_ID'])
                data.append(color_data['DATA'])

In [ ]:
set(blocks)

In [ ]:
from nbtlib import nbt,tag
schem = nbt.load('data/original')
out_path = os.path.join('output',OUTPUT_FILE)
os.makedirs('output',exist_ok=True)
output(out_path,schem)